<a href="https://colab.research.google.com/github/surajsrivathsa/thesis_comics_search_xai/blob/main/feature_extraction/jupyter_notebooks/BookNLP_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble

In [ ]:
!pip install booknlp
!pip install spacy
!python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 7.8 MB/s 
     |████████████████████████████████| 4.4 MB 35.1 MB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.2 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
  Created wheel for booknlp: filename=booknlp-1.0.7-py3-none-any.whl size=2420766 sha256=87169697e8c0650efd1c4a5bdfb28bac62901d6ff7252f0f4a80e5bf2fa8256c
  Stored in directory: /root/.cache/pip/wheels/3e/79/30/e49074dd91db7434da782e5d48e44bdabe066c07adf465f376
Successfully built booknlp
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-w

In [ ]:
from booknlp.booknlp import BookNLP

using device cpu


In [ ]:
import pandas as pd
import numpy as np
import torch
import pickle
import json
from collections import Counter
import torch
import os, sys, glob, re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

thesis_filepath = '/content/drive/MyDrive/suraj/thesis'

dataset_path = os.path.join(thesis_filepath, 'dataset', 'emoreccom_public_train')
os.chdir(dataset_path)

Mounted at /content/drive


# preprocess text per book

In [ ]:
raw_text_df = pd.read_csv('combined_text_img_df.csv')
print(raw_text_df.shape)
raw_text_df.head()

(51218, 10)


,comic_no,page_no,panel_no,text,concatenated_text,fastpunct_text,decontracted_text,text_clean,image_id,img_file_path
0,3451,2,0,['mercy but the boy he tries to save is too ho...,mercy but the boy he tries to save is too hot ...,"['Mercy, but the boy he tries to save is too h...","['mercy , but the boy he tries to save is too ...","['mercy , but the boy he tries to save is too ...",3451_2_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...
1,3451,3,0,"['nsion several miles from the grove, althy un...","nsion several miles from the grove , althy un...","['Cinder several miles from the grove, along R...","['cinder several miles from the grove , along ...","['cinder several miles from the grove , along ...",3451_3_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...
2,3451,3,1,"['e e owwwi', 'randy expresses his dislike ey ...",e - e - owwwi. randy expresses his dislike ey ...,"['E e owww.i.', 'Randy expresses his dislike a...","['e e owww .i .', 'randy expresses his dislike...","['e e owww .i .', 'randy expresses his dislike...",3451_3_1,/content/drive/MyDrive/suraj/thesis/raw_panel_...
3,3451,3,2,"['so. take one yourself, meadowsd']","so ... take one yourself , meadowsd","['So take one yourself, meadowsd.']","['so take one yourself , meadowsd .']","['so take one yourself , meadowsd .']",3451_3_2,/content/drive/MyDrive/suraj/thesis/raw_panel_...
4,3451,3,3,"['as meadows flees from the room, randy fills ...","as meadows flees from the room , randy fills ...","['As Meadows flees from the room, Randy fills ...","['as meadows flees from the room , randy fills...","['as meadows flees from the room , randy fills...",3451_3_3,/content/drive/MyDrive/suraj/thesis/raw_panel_...


In [ ]:
comic_info_df = pd.read_csv('comicnum_to_book_title.csv')
print(comic_info_df.shape)
comic_info_df.dropna(subset=['Book Title', 'Vol'], how='all', inplace=True)
print(comic_info_df.shape)
comic_info_df.head()

(491, 8)
(165, 8)


,comic_no,Book Title,Vol,Issue,Year,Month,genre,link
0,3451,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
1,3452,Dick Cole - Blue Bolt,3.0,1.0,1949,Jun,superhero|vigilante,NaN
2,3453,Dick Cole - Blue Bolt,5.0,3.0,NaN,Dec,superhero|vigilante,NaN
3,3454,Dick Cole - Blue Bolt,2.0,7.0,NaN,Dec,superhero|vigilante,NaN
4,3455,Dick Cole - Blue Bolt,5.0,7.0,NaN,NaN,superhero|vigilante,NaN


In [ ]:
comic_info_df.tail()

,comic_no,Book Title,Vol,Issue,Year,Month,genre,link
160,3619,Kaanga - Witch Queen of the Hairy Ones,NaN,93.0,1947,Sep,jungle|animal|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
161,3620,kaanga - Vendetta of the Free Tribes,NaN,92.0,1947,Aug,jungle|animal|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
162,3621,JoJo - The Mountain of Skulls,NaN,8.0,1947,Nov,jungle|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
163,3622,JoJo - The Copy-Mad Killers,NaN,28.0,1949,Jun,jungle|adventure,https://digitalcomicmuseum.com/index.php?dlid=...
164,3623,Jumbo Comics - Sheena - Tigerman terror,NaN,26.0,1941,Apr,jungle|adventure|female|scifi|children,https://digitalcomicmuseum.com/index.php?dlid=...


In [ ]:
merged_df = pd.merge(right =comic_info_df, left=raw_text_df, left_on='comic_no', right_on='comic_no', how='inner')
print(merged_df.shape)
merged_df.head()

(50778, 17)


,comic_no,page_no,panel_no,text,concatenated_text,fastpunct_text,decontracted_text,text_clean,image_id,img_file_path,Book Title,Vol,Issue,Year,Month,genre,link
0,3451,2,0,['mercy but the boy he tries to save is too ho...,mercy but the boy he tries to save is too hot ...,"['Mercy, but the boy he tries to save is too h...","['mercy , but the boy he tries to save is too ...","['mercy , but the boy he tries to save is too ...",3451_2_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
1,3451,3,0,"['nsion several miles from the grove, althy un...","nsion several miles from the grove , althy un...","['Cinder several miles from the grove, along R...","['cinder several miles from the grove , along ...","['cinder several miles from the grove , along ...",3451_3_0,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
2,3451,3,1,"['e e owwwi', 'randy expresses his dislike ey ...",e - e - owwwi. randy expresses his dislike ey ...,"['E e owww.i.', 'Randy expresses his dislike a...","['e e owww .i .', 'randy expresses his dislike...","['e e owww .i .', 'randy expresses his dislike...",3451_3_1,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
3,3451,3,2,"['so. take one yourself, meadowsd']","so ... take one yourself , meadowsd","['So take one yourself, meadowsd.']","['so take one yourself , meadowsd .']","['so take one yourself , meadowsd .']",3451_3_2,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN
4,3451,3,3,"['as meadows flees from the room, randy fills ...","as meadows flees from the room , randy fills ...","['As Meadows flees from the room, Randy fills ...","['as meadows flees from the room , randy fills...","['as meadows flees from the room , randy fills...",3451_3_3,/content/drive/MyDrive/suraj/thesis/raw_panel_...,Blue Bolt,10.0,2.0,1949,Sep,superhero|vigilante,NaN


In [ ]:
merged_df.tail()

In [ ]:
print(merged_df.shape)
merged_df.iloc[0,7]

(50778, 17)


"['mercy , but the boy he tries to save is too hot to handle .', 'on summer vacation from farr military aca dem y simba karno and dick camp out in a redwoo grove in california .', 'great idea , coming out here , dick ! it is beautiful !', 'blue button , vol . 10 , no . 2 , september october , 1949 , publish ed bi monthly by the premium group comics , a of curtis circulation company , p . 0 . box 1198 , independence square , philadelphia , pa . offices , we .']"

In [ ]:
def strip_brackets(text_clean):
  text_clean_new = ''
  text_clean_new  = text_clean.replace('[', '')
  text_clean_new  = text_clean_new.replace(']', '')
  text_clean_new  = text_clean_new.replace("'", '')
  text_clean_new  = text_clean_new.replace(".,", '.')

  return text_clean_new

In [ ]:
len(merged_df['comic_no'].unique())

165

In [ ]:
text_per_book_df = merged_df.groupby(['comic_no','Book Title']).agg({'text_clean': lambda x: ' '.join(x)}).reset_index()
text_per_book_df['text_clean_new'] = text_per_book_df['text_clean'].apply(strip_brackets)
text_per_book_df['book_title_clean'] = text_per_book_df['Book Title'].apply(lambda x: re.sub('\W+','_', x.lower() ))
text_per_book_df['unique_id'] = text_per_book_df['comic_no'].astype(str) + '_' + text_per_book_df['book_title_clean']
print(text_per_book_df.shape)
text_per_book_df.head()

(165, 6)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt


In [ ]:
def save_dialogue_as_text(row):

  with open(row['unique_id']+".txt", "w") as text_file:
    text_file.write(row['text_clean_new'])

  row['dialogue_text_filepath'] = os.path.join(dataset_path, row['unique_id']+".txt")
  return row

In [ ]:
text_per_book_df = text_per_book_df.apply(save_dialogue_as_text, axis=1)

In [ ]:
text_per_book_df.head()

,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
text_per_book_df.dtypes

comic_no                   int64
Book Title                object
text_clean                object
text_clean_new            object
book_title_clean          object
unique_id                 object
dialogue_text_filepath    object
dtype: object

In [ ]:
text_per_book_df.to_csv('booknlp_text_per_book_df.csv', index=False)

# Initialize book nlp pipeline

In [ ]:
model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big", 
	}

In [ ]:
booknlp=BookNLP("en", model_params)

{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
downloading entities_google_bert_uncased_L-6_H-768_A-12-v1.0.model
downloading coref_google_bert_uncased_L-12_H-768_A-12-v1.0.model
downloading speaker_google_bert_uncased_L-12_H-768_A-12-v1.0.1.model


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

--- startup: 67.821 seconds ---


# Run Book NLP Process to extract all entities

In [ ]:
text_per_book_df = pd.read_csv('booknlp_text_per_book_df.csv')
text_per_book_df = text_per_book_df[text_per_book_df['comic_no'] > 3520]
print(text_per_book_df.shape)
text_per_book_df.head()

(95, 7)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
70,3521,Daring Adventures - Dynamic man meets the litt...,['suched the grisly answer although knew he wo...,suched the grisly answer although knew he woul...,daring_adventures_dynamic_man_meets_the_little...,3521_daring_adventures_dynamic_man_meets_the_l...,/content/drive/MyDrive/suraj/thesis/dataset/em...
71,3522,Dynamic Man- The Echo,['comic .'] ['the dreams of dozens of poor boy...,comic . the dreams of dozens of poor boys were...,dynamic_man_the_echo,3522_dynamic_man_the_echo,/content/drive/MyDrive/suraj/thesis/dataset/em...
72,3523,Dynamic Man - man on the run,"[""what defiant urges zao a s betzine , the unw...","""what defiant urges zao a s betzine , the unwi...",dynamic_man_man_on_the_run,3523_dynamic_man_man_on_the_run,/content/drive/MyDrive/suraj/thesis/dataset/em...
73,3524,The Eagle - Monster of tropics,['before the nazis could open fire on the red ...,before the nazis could open fire on the red cr...,the_eagle_monster_of_tropics,3524_the_eagle_monster_of_tropics,/content/drive/MyDrive/suraj/thesis/dataset/em...
74,3525,Exciting Comics - Black terror - Malay Menace,"['eating the black terror .', 'black terror , ...","eating the black terror . black terror , immed...",exciting_comics_black_terror_malay_menace,3525_exciting_comics_black_terror_malay_menace,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
def extract_information_from_dialogues(row):
  print(str(row.name) + ' ' + row['unique_id'])
  inputFile = row['dialogue_text_filepath']
  outputDir = row['unique_id'] + "/"
  idd = row['unique_id']
  booknlp.process(inputFile, outputDir, idd)
  print(' ============= ============== ============== ============ ')
  print()

  return row


In [ ]:
text_per_book_df = text_per_book_df.apply(extract_information_from_dialogues, axis=1)

70 3521_daring_adventures_dynamic_man_meets_the_little_people
--- spacy: 0.540 seconds ---
--- entities: 12.406 seconds ---
--- quotes: 0.008 seconds ---
--- attribution: 2.267 seconds ---
--- name coref: 0.007 seconds ---
--- coref: 19.672 seconds ---
--- TOTAL (excl. startup): 35.611 seconds ---, 3827 words
 ============= ============== ============== ============ 

71 3522_dynamic_man_the_echo
--- spacy: 1.033 seconds ---
--- entities: 29.585 seconds ---
--- quotes: 0.017 seconds ---
--- attribution: 4.454 seconds ---
--- name coref: 0.023 seconds ---
--- coref: 36.634 seconds ---
--- TOTAL (excl. startup): 72.405 seconds ---, 8120 words
 ============= ============== ============== ============ 

72 3523_dynamic_man_man_on_the_run
--- spacy: 0.731 seconds ---
--- entities: 17.556 seconds ---
--- quotes: 0.013 seconds ---
--- attribution: 6.843 seconds ---
--- name coref: 0.016 seconds ---
--- coref: 26.047 seconds ---
--- TOTAL (excl. startup): 51.269 seconds ---, 5671 words
 ======

--- spacy: 1.032 seconds ---
--- entities: 10.091 seconds ---
--- quotes: 0.018 seconds ---
--- attribution: 0.837 seconds ---
--- name coref: 0.013 seconds ---
--- coref: 6.804 seconds ---
--- TOTAL (excl. startup): 19.103 seconds ---, 8083 words


# Evaluate output

In [ ]:
text_per_book_df = pd.read_csv('booknlp_text_per_book_df.csv')
print(text_per_book_df.shape)
text_per_book_df.head()

(165, 7)


,comic_no,Book Title,text_clean,text_clean_new,book_title_clean,unique_id,dialogue_text_filepath
0,3451,Blue Bolt,"['mercy , but the boy he tries to save is too ...","mercy , but the boy he tries to save is too ho...",blue_bolt,3451_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
1,3452,Dick Cole - Blue Bolt,"['ld cap hawkins , ill .', 'blue bolt , sub ze...","ld cap hawkins , ill . blue bolt , sub zero ed...",dick_cole_blue_bolt,3452_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
2,3453,Dick Cole - Blue Bolt,"['eating blue bol .', 'december .'] ['some sho...",eating blue bol . december . some short sighte...,dick_cole_blue_bolt,3453_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
3,3454,Dick Cole - Blue Bolt,"['blue , bolt , edison bell is dick cole .'] [...","blue , bolt , edison bell is dick cole . vol ....",dick_cole_blue_bolt,3454_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...
4,3455,Dick Cole - Blue Bolt,['come and learn .'] ['the basketball season i...,come and learn . the basketball season is in f...,dick_cole_blue_bolt,3455_dick_cole_blue_bolt,/content/drive/MyDrive/suraj/thesis/dataset/em...


In [ ]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

In [ ]:
data=proc("3452_dick_cole_blue_bolt/3452_dick_cole_blue_bolt.book")

In [ ]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

In [ ]:
dick_cole_blue_bolt_entity_3452_df = pd.read_csv("./3452_dick_cole_blue_bolt/3452_dick_cole_blue_bolt.entities", sep='\t')
print(dick_cole_blue_bolt_entity_3452_df.shape)
dick_cole_blue_bolt_entity_3452_df.head(100)

(1661, 6)


,COREF,start_token,end_token,prop,cat,text
0,50,0,12,NOM,PER,"ld cap hawkins , ill . blue bolt , sub zero ed..."
1,51,22,22,PRON,PER,your
2,52,34,34,PRON,PER,his
3,8,55,57,PROP,PER,john dal marines
4,53,59,59,PRON,PER,us
...,...,...,...,...,...,...
95,8,464,464,PRON,PER,he
96,66,466,466,NOM,PER,indians
97,8,469,469,PRON,PER,he
98,66,471,471,NOM,PER,indians


In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

8 43 john he/him/his

	poss	3 father
	poss	1 shoulder
	poss	1 work
	poss	1 own
	poss	1 sons
	poss	1 boys

	agent	1 told
	agent	1 comes
	agent	1 studied
	agent	1 tell


	mod	5 private
	mod	1 married
	mod	1 sort
	mod	1 hobby

10 25 dick he/him/his

	poss	1 face
	poss	1 pursuers
	poss	1 puckers
	poss	1 head

	agent	1 starts
	agent	1 picks
	agent	1 eudes
	agent	1 hides
	agent	1 joins
	agent	1 used
	agent	1 accomplished
	agent	1 typifies
	agent	1 reach
	agent	1 started

	patient	1 fix

	mod	2 right

14 21 eddie he/him/his


	agent	2 has
	agent	1 take
	agent	1 sets
	agent	1 flattens
	agent	1 finds
	agent	1 hears
	agent	1 discovered

	patient	1 sends
	patient	1 see
	patient	1 got
	patient	1 find

	mod	1 about

42 14 tito he/him/his

	poss	1 pursler
	poss	1 guns

	agent	1 held
	agent	1 smi
	agent	1 get
	agent	1 go

	patient	1 head
	patient	1 jump

	mod	1 bandit
	mod	1 smarter

19 11 krubb he/him/his

	poss	1 men

	agent	1 takes
	agent	1 dropped
	agent	1 walk
	agent	1 says
	agent	1 leave
	agent

In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

5 29 steve he/him/his

	poss	2 pursuers
	poss	1 ick
	poss	1 squadron
	poss	1 tv
	poss	1 course
	poss	1 wings
	poss	1 intentions

	agent	1 escorted
	agent	1 sparked
	agent	1 given
	agent	1 return
	agent	1 takes
	agent	1 reversed
	agent	1 striving
	agent	1 wags
	agent	1 tries
	agent	1 knocked

	patient	1 shot
	patient	1 forced


7 2 randy oconnor he/him/his

	poss	1 allies

	agent	1 observed





In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

5 22 buster crabbe he/him/his


	agent	1 got
	agent	1 showing
	agent	1 git
	agent	1 taking
	agent	1 ai
	agent	1 put
	agent	1 sees
	agent	1 draws
	agent	1 pull
	agent	1 go

	patient	1 killed
	patient	1 silence
	patient	1 learned
	patient	1 beat
	patient	1 get
	patient	1 got

	mod	1 friend
	mod	1 dead

10 5 minnie boo he/him/his


	agent	1 had
	agent	1 wuz
	agent	1 tromped
	agent	1 massacred



7 4 benson he/him/his

	poss	1 gang

	agent	1 call

	patient	1 locked
	patient	1 tell


8 3 .the coroanches they/them/their


	agent	1 knew



6 2 jason he/him/his







In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

6 13 bryan craig he/him/his

	poss	1 alley
	poss	1 ag
	poss	1 light

	agent	1 like
	agent	1 did
	agent	1 hateem

	patient	1 lost
	patient	1 get
	patient	1 tell
	patient	1 atomize
	patient	1 gaught


13 9 miss tracy she/her

	poss	1 soldier

	agent	1 avenge

	patient	1 tell

	mod	2 spy
	mod	1 help
	mod	1 human

4 8 gordon he/him/his


	agent	1 keep



14 8 krain he/him/his


	agent	1 turns
	agent	1 decides
	agent	1 attracts
	agent	1 hidden
	agent	1 snaps
	agent	1 lands



10 3 captain schenge he/him/his

	poss	2 belt

	agent	1 takes



5 2 trevor r he/him/his

	poss	1 assistant




15 2 frank he/him/his



	patient	1 save




In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()

3 18 brenda she/her

	poss	1 plane
	poss	1 power

	agent	1 recovers
	agent	1 has
	agent	1 drink
	agent	1 got
	agent	1 find
	agent	1 rested
	agent	1 hear

	patient	1 ntranced

	mod	1 lucky
	mod	1 startled

8 6 tom he/him/his


	agent	1 soars
	agent	1 got

	patient	1 dragging


7 5 miss starr she/her

	poss	1 innocence


	patient	1 stop


10 3 guido marcanni he/him/his





